<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Catherine Benavides
- Nombre de alumno 2: Fernanda Saavedra


### **Link de repositorio de GitHub:** https://github.com/FernandaSD/MDS7202

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import Ridge

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
df_players = pd.read_csv('/content/drive/MyDrive/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]

Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
# Cantidad de jugadores seleccionados y no seleccionados
print("Cantidad de jugadores no seleccionados:", df_players['National_Position'].isna().sum())
print("Cantidad de jugadores seleccionados:", df_players.shape[0]-df_players['National_Position'].isna().sum())

Cantidad de jugadores no seleccionados: 16513
Cantidad de jugadores seleccionados: 1075


In [5]:
# Variable target
df_players['National_Position'] = df_players['National_Position'].isna().apply(lambda x: 0 if x == True else 1)

Se tienen 1075 futbolistas clasificados y 16513 futbolístas no clasificado. Se comenta que estas proporciones coinciden con lo esperado, ya que el porcentaje de jugadores que logran ser seleccionados nacionales es bajo. Por otra parte, como las clases están desbalanceadas podrían haber problemas más adelante en los modelos.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

**Preprocesamiento de variables de variables**

- Para variables categóricas se utilizará OneHotEncoder, dado que se trata de variables nominales.
- En el caso de las variables numéricas se aplica MinMaxScaler. Se consideró relevante el uso de escaladores ya que las variables tienen distintos rangos lo cual podría sesgar el análisis. Además, en el EDA no se observan variables con valores atipícos muy extremos, por lo que se prefirió usar un escalador fácilmente interpretable y que mantenga la distribución de los datos.
- Por otro lado, 'National_Position' no necesita ser transformada al ser la variable objetivo por lo que se utiliza passthrough.

Finalmente, se comenta que se eliminó la variable "Name" ya que tiene una cardinalidad muy alta (más de 95% de la cardinalidad de la base de datos) y porque no se espera que haya una correlación muy alta con esta variable y el hecho de que un jugador sea clasificiado nacional. 

In [6]:
# Datos Categóricos
cat_cols = ['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate']
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Datos numéricos
num_cols = ['Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
num_transformer = MinMaxScaler()

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('cat', cat_transformer, cat_cols),
        ('num', num_transformer, num_cols)],
    remainder='passthrough', 
    sparse_threshold = 0)

Ahora, de acuerdo a la guía, se cuenta con más de 50 registros y se está prediciendo una categoría. Luego, se tiene que se cuenta con la data etiquetada y con menos de 100.000 registros. Dado lo anterior, los modelos a implementar son Linear SVC, KNeighborsClassifier y RandomForestCLassifier.

In [7]:
# Pipeline
pipeline_lsvc = Pipeline(steps=[('preprocessor', preprocessor), 
 ('linearsvc', LinearSVC(random_state=0))])

pipeline_knc = Pipeline(steps=[('preprocessor', preprocessor), 
 ('knc', KNeighborsClassifier(n_neighbors=3))])

pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), 
 ('rf', RandomForestClassifier(max_depth=2, random_state=0))])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [8]:
# Separación del dataset
df_players.dropna(inplace = True)

X = df_players[['Nationality', 'Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']]

y = df_players['National_Position']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.3, random_state=0)

En las variables independientes (X) no se incluye 'Name' ya que se considera que no aporta a la clasificación.

In [9]:
# Entrenamiento y métricas de evaluación
pipeline_lsvc.fit(X_train, y_train)
y_pred_lsvc = pipeline_lsvc.predict(X_test)
print(classification_report(y_test, y_pred_lsvc))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.96      0.99      0.97      4954
           1       0.70      0.33      0.45       323

    accuracy                           0.95      5277
   macro avg       0.83      0.66      0.71      5277
weighted avg       0.94      0.95      0.94      5277



In [10]:
# Entrenamiento y métricas de evaluación
pipeline_knc.fit(X_train, y_train)
y_pred_knc = pipeline_knc.predict(X_test)
print(classification_report(y_test, y_pred_knc))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4954
           1       0.47      0.15      0.23       323

    accuracy                           0.94      5277
   macro avg       0.71      0.57      0.60      5277
weighted avg       0.92      0.94      0.92      5277



In [11]:
# Entrenamiento y métricas de evaluación
pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.00      0.00      0.00       323

    accuracy                           0.94      5277
   macro avg       0.47      0.50      0.48      5277
weighted avg       0.88      0.94      0.91      5277



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?**

La clase que indica que un jugador es seleccionado nacional (valor 1) y en esta se pueden observar bajos resultados en las métricas. El accuracy indica qué tan bien el modelo clasifica correctamente a los jugadores, tanto a los seleccionados como los no seleccionados y es la métrica que mejores resultados obtiene, variando entre 0.94 y 0.95. Esto se debe a que considera tanto la clase mayoritaria (No seleccionados) como la clase minoritaria (Seleccionados). 

La precisión muestra qué tan confiable es el modelo al predecir si un jugador es seleccionado nacional, es decir, cuántas de las predicciones positivas son realmente correctas. Los resultados obtenidos para esta métrica son sustancialmente menores, con resultados que varían entre 0.7 y 0. 

El recall muestra qué tan sensible es el modelo para detectar a los jugadores seleccionados, es decir, cuántos de los jugadores seleccionados se clasifican correctamente. Esta es la métrica con peor desempeño, con valores que van entre 0 y 0.33. 

Estos resultados coinciden con lo esperado teóricamente, dado que la clase Seleccionado es minoritaria, por lo que los clasificadores están sesgados a clasificar los datos en la otra clase.

El modelo con mejores resultados en Linear SVC, con resultados de 0.70, 0.33 y 0.95 para los valores de precision, recall y accuracy respectivamente. Luego viene K-Neighbors Classifier con 0.47, 0.15 y 0.94, respectivamente. Finalmente está Random Forest con métricas de 0, 0 y 0.94 respectivamente. De estos resultado se observa que Linear SVC es el modelo que mejor se adecua a estos datos, al obtener los mejores resultados para la clase negariva. 




**¿Podrían mejorarse los resultados?, ¿Cómo?**

Los resultados pueden ser mejorables buscando balancear las clases que es el principal problema que tiene este dataset. Para ello se pueden utilizar técnicas como undersampling y oversampling. Además, se podrían optimizar los hiperparámetros de los modelos, utilizando GridSearch. Por otro lado, se podría evaluar la posibilidad de incorporar datos adicionales a la clase minoritaria. Otra solución sería alterar el peso que tiene cada clase en el entrenamiento


**¿Influye la cantidad de ejemplos por clase?**

De acuerdo a lo mencionado anteriormente, el desbalance de las clases induce sesgos en el modelo, favoreciendo que este clasifique los datos en la clase predominante sin importar sus características. 

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [12]:
# Se copia el dataset
df = df_players.copy()

# Se reemplazan los valores de Club_Position
df['Club_Position'].replace(['ST', 'CF'] , 'ataque', inplace=True)
df['Club_Position'].replace(['RW', 'CAM', 'LW'] , 'central_ataque', 
                            inplace=True)
df['Club_Position'].replace(['RM', 'CM', 'LM']  , 'central', inplace=True)
df['Club_Position'].replace(['RWB', 'CDM', 'LWB']  , 'central_defensa', 
                                    inplace=True)
df['Club_Position'].replace(['RB', 'CB', 'LB']  , 'defensa', inplace=True)
df['Club_Position'].replace(['GK']  , 'arquero', inplace=True)

# Se filtra el dataset para descartar Sub y Res
df = df[df['Club_Position'].isin(['ataque', 'central_ataque', 'central', 
                                  'central_defensa', 'defensa', 'arquero'])]

In [13]:
# Cantidad de jugadores por clase
df['Club_Position'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

Nuevamente se puede observar un desbalance entre las clases, donde se tiene que 'defensa' es por lejos más grande que 'central_defensa', que es la clase con la menor cantidad de ejemplos.

In [14]:
# Datos Categóricos (Se omite la variable Name ya que se considera que no aporta 
# para la clasificación)
cat_cols_2 = ['Nationality', 'Preffered_Foot', 'Work_Rate']
cat_transformer_2 = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Datos numéricos
num_cols_2 = ['Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
num_transformer_2 = MinMaxScaler()

# ColumnTransformer
preprocessor_2 = ColumnTransformer(
    transformers=[('cat', cat_transformer_2, cat_cols_2),
        ('num', num_transformer_2, num_cols_2)],
    remainder='passthrough', 
    sparse_threshold = 0)

In [15]:
# Pipeline
pipeline_lsvc_2 = Pipeline(steps=[('preprocessor', preprocessor_2), 
 ('linearsvc', LinearSVC(random_state=0))])

pipeline_knc_2 = Pipeline(steps=[('preprocessor', preprocessor_2), 
 ('knc', KNeighborsClassifier(n_neighbors=3))])

pipeline_rf_2 = Pipeline(steps=[('preprocessor', preprocessor_2), 
 ('rf', RandomForestClassifier(max_depth=2, random_state=0))])

In [16]:
# Separación del dataset
X_2 = df[['Nationality', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']]

y_2 = df['Club_Position']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, stratify=y_2, 
                                                    test_size=0.3, random_state=1)

In [17]:
# Entrenamiento y métricas de evaluación
pipeline_lsvc_2.fit(X_train_2, y_train_2)
y_pred_lsvc_2 = pipeline_lsvc_2.predict(X_test_2)
print(classification_report(y_test_2, y_pred_lsvc_2))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


                 precision    recall  f1-score   support

        arquero       1.00      0.99      1.00       190
         ataque       0.78      0.87      0.82       129
        central       0.55      0.58      0.57       272
 central_ataque       0.43      0.30      0.35       174
central_defensa       0.62      0.21      0.31        63
        defensa       0.80      0.95      0.87       354

       accuracy                           0.73      1182
      macro avg       0.70      0.65      0.65      1182
   weighted avg       0.71      0.73      0.71      1182



In [18]:
# Entrenamiento y métricas de evaluación
pipeline_knc_2.fit(X_train_2, y_train_2)
y_pred_knc_2 = pipeline_knc_2.predict(X_test_2)
print(classification_report(y_test_2, y_pred_knc_2))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


                 precision    recall  f1-score   support

        arquero       1.00      0.99      1.00       190
         ataque       0.48      0.55      0.51       129
        central       0.40      0.46      0.43       272
 central_ataque       0.38      0.29      0.33       174
central_defensa       0.29      0.13      0.18        63
        defensa       0.72      0.76      0.74       354

       accuracy                           0.60      1182
      macro avg       0.54      0.53      0.53      1182
   weighted avg       0.59      0.60      0.59      1182



In [19]:
# Entrenamiento y métricas de evaluación
pipeline_rf_2.fit(X_train_2, y_train_2)
y_pred_rf_2 = pipeline_rf_2.predict(X_test_2)
print(classification_report(y_test_2, y_pred_rf_2))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.00      0.00      0.00       129
        central       0.42      0.73      0.53       272
 central_ataque       0.00      0.00      0.00       174
central_defensa       0.00      0.00      0.00        63
        defensa       0.68      0.99      0.81       354

       accuracy                           0.63      1182
      macro avg       0.35      0.45      0.39      1182
   weighted avg       0.46      0.63      0.53      1182



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Se puede observar que en todos los modelos la clase 'arquero' fue predicha completamente correcta, mientras que 'central_defensa' es la que obtuvo peores resultados. Esto se puede relacionar con la baja cantidad de ejemplos que presentaba la clase central defensa. Con respecto a la clase "arquero" se comenta que obtiene métricas perfectas, lo cual indica que los modelos están sobre-ajustados.

De los modelos se comenta que Random Forest vuelve a tener los peores resultados y Linear SVC los mejores resultados. 

Igual que el caso anterior, los resultados podrían mejorarse si se aplican técnicas de balanceo de clases y optimización de hiperparámetros de los modelos.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [20]:
# Se carga el dataset
df_salaries = pd.read_csv('/content/drive/MyDrive/salarios.csv', index_col = 0)
df_salaries.head()

,Player,Weekly Salary
1528,Lionel Messi,1538905.0
1006,Cristiano Ronaldo,1248536.0
1529,Antoine Griezmann,997032.0
2004,Neymar,797726.0
1530,Gareth Bale,607706.0


In [21]:
# Se unen los dataset
df_3 = df_players.copy()
df_3 = df_3.merge(df_salaries, left_on = 'Name', right_on = 'Player', how = 'inner')
df_3.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Player,Weekly Salary
0,Cristiano Ronaldo,Portugal,1,LW,185,80,Right,32,High / Low,4,...,85,92,93,90,81,76,85,88,Cristiano Ronaldo,1248536.0
1,Lionel Messi,Argentina,1,RW,170,72,Left,29,Medium / Medium,4,...,71,85,95,88,89,90,74,85,Lionel Messi,1538905.0
2,Neymar,Brazil,1,LW,174,68,Right,25,High / Medium,5,...,62,78,89,77,79,84,81,83,Neymar,797726.0
3,Luis Suárez,Uruguay,1,ST,182,85,Right,30,High / Medium,4,...,77,87,94,86,86,84,85,88,Luis Suárez,508923.0
4,Manuel Neuer,Germany,1,GK,193,92,Right,31,Medium / Medium,4,...,25,25,13,16,14,11,47,11,Manuel Neuer,326233.0


In [22]:
# Dimensiones del dataset
df_3.shape

(1866, 41)

In [23]:
# Tipos de columnas
df_3.dtypes

Name                  object
Nationality           object
National_Position      int64
Club_Position         object
Height                 int64
Weight                 int64
Preffered_Foot        object
Age                    int64
Work_Rate             object
Weak_foot              int64
Skill_Moves            int64
Ball_Control           int64
Dribbling              int64
Marking                int64
Sliding_Tackle         int64
Standing_Tackle        int64
Aggression             int64
Reactions              int64
Interceptions          int64
Vision                 int64
Composure              int64
Crossing               int64
Short_Pass             int64
Long_Pass              int64
Acceleration           int64
Speed                  int64
Stamina                int64
Strength               int64
Balance                int64
Agility                int64
Jumping                int64
Heading                int64
Shot_Power             int64
Finishing              int64
Long_Shots    

In [24]:
# Datos Categóricos (Se omite la variable Name ya que se considera que no aporta 
# para la clasificación)
cat_cols_3 = ['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate']
cat_transformer_3 = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Datos numéricos
num_cols_3 = ['Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'National_Position']
num_transformer_3 = MinMaxScaler()

# ColumnTransformer
preprocessor_3 = ColumnTransformer(
    transformers=[('cat', cat_transformer_3, cat_cols_3),
        ('num', num_transformer_3, num_cols_3)],
    remainder='passthrough', 
    sparse_threshold = 0)

Ahora, de acuerdo a la guía, se cuenta con más de 50 registros, no se está prediciendo una categoría, pero sí una cantidad. Luego, se tiene que hay menos de 100.000 registros. Dado lo anterior, los modelos a implementar RidgeRegression, SVR(kernel=linear) y RandomForestRegressor.

In [25]:
# Pipeline
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor_3), 
 ('linearsvc', Ridge())])

pipeline_svr = Pipeline(steps=[('preprocessor', preprocessor_3), 
 ('knc', SVR(kernel = 'linear'))])

pipeline_rfr = Pipeline(steps=[('preprocessor', preprocessor_3), 
 ('rf', RandomForestRegressor(max_depth=2, random_state=0))])

In [26]:
# Separación del dataset
X_3 = df_3[['Nationality', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'National_Position',
       'Club_Position']]

y_3 = df_3['Weekly Salary']

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.3, random_state=0)

In [27]:
# Entrenamiento y métrica de evaluación
pipeline_ridge.fit(X_train_3, y_train_3)
y_pred_ridge = pipeline_ridge.predict(X_test_3)
r2_score(y_test_3, y_pred_ridge)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.3267900639430309

In [28]:
# Entrenamiento y métrica de evaluación
pipeline_svr.fit(X_train_3, y_train_3)
y_pred_svr = pipeline_svr.predict(X_test_3)
r2_score(y_test_3, y_pred_svr)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


-0.11983877945138688

In [29]:
# Entrenamiento y métrica de evaluación
pipeline_rfr.fit(X_train_3, y_train_3)
y_pred_rfr = pipeline_rfr.predict(X_test_3)
r2_score(y_test_3, y_pred_rfr)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.48055834519045904

El coeficiente $R^2$ se interpreta como la varianza que es explicada por el modelo; entre más cercano sea esta valor a uno, mejor las variables independientes explican a la varianza de la varible dependiente.

Los modelos Ridge y SVR no logran superar el 0.35 para $R^2$, incluso SVR obtuvo un coeficiente negativo. Por otra parte, RandomForestRegressor obtuvo un coeficiente de 0.48 aproximadamente superando el mínimo solicitado. 

Para mejorar los resultados se propone buscar nuevas variables que estén más relacionadas con el ingreso de un jugador de fútbol. Se tienen múltiples variables relacionadas con el desempeño como jugador, pero no tantas relacionadas con su capacidad para generar ingresos. Algunas variables de interés podría ser el país de residencia, edad del jugador, club en el que entrena, etc. Ahora bien, si esto no llegara a ser suficiente sería bueno ampliar la base de datos, especialemente para los niveles de ingresos menos representados.

# Conclusión

Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>